In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
import datetime
import random
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, 'L' code_mode_exercice  from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [102]:
def get_pa(year, specialite, is_medecin):
    if is_medecin:
        sql = f"""
select pa.id, pa.inpp, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
--and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id, ps.code_mode_exercice
"""
    else:
        sql = f"""
select pa.id, pa.inpp, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
--and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id, ps.code_mode_exercice
"""    
    print(f"Quering PA for year {year} and specialite {specialite} for is_medecin={is_medecin}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [103]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite, specialite<21)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [5]:
def get_pop_iris(year):
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

In [6]:
def get_iris_matrix(time: int, time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m



In [107]:
specialite=10
year=23
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-0.12
print(year, specialite, source, time, time_type, accessibilite_exp)
debug=61210000
writer = pd.ExcelWriter(f"out/debug_{debug}_{year}_{specialite}_{time}.xlsx")

23 21 PA 30 HC -0.12


In [8]:
# Very long
iris_matrix = get_iris_matrix(time, time_type)
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= 30) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,10010000,10010000,0,0,0,0,10010000
1,10010000,10050000,21,24,24,24,10050000
2,10010000,10210000,25,30,30,30,10210000
3,10010000,10250000,22,28,30,28,10250000
4,10010000,10260000,20,25,27,25,10260000
...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000
19272671,2023650000,2022510000,10,15,15,15,2022510000
19272672,2023650000,2022770000,16,28,28,28,2022770000
19272673,2023650000,2023650000,0,0,0,0,2023650000


In [105]:
pop_iris = get_pop_iris(year)
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,821300000,821300000,82130,C,170.000000,1.024096,3.072289,8.192771,5.120482,4.096386,28.674699,33.795181,26.626506,44.036145,15.361446
1,811540000,811540000,81154,C,80.000000,2.105263,3.157895,5.263158,4.210526,2.105263,4.210526,17.894737,17.894737,16.842105,6.315789
2,840060000,840060000,84006,C,71.000000,2.028571,3.042857,9.128571,3.042857,2.028571,16.228571,12.171429,9.128571,6.085714,8.114286
3,821570000,821570000,82157,C,107.000000,4.734513,3.787611,4.734513,5.681416,7.575221,16.097345,18.938053,17.991150,22.725664,4.734513
4,840070113,840070113,84007,A,2744.168077,66.531592,76.166839,91.027356,186.769463,466.091795,711.988855,459.891850,281.679718,253.076015,150.944594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48550,940460103,940460103,94046,A,1773.227226,43.209398,54.214737,66.914813,90.574673,141.873872,412.719594,339.034379,197.545292,327.448839,99.691628
48551,940460109,940460109,94046,A,2474.409868,60.984598,64.709729,183.003802,264.548488,197.623284,425.604137,527.371166,305.141054,330.464804,114.958805
48552,940460202,940460202,94046,A,2435.011871,90.000439,77.000375,125.000609,198.000965,427.002082,680.003315,670.003266,145.000707,17.000083,6.000029
48553,940540105,940540105,94054,A,2075.664303,59.233979,69.397063,104.184780,218.764070,149.651718,291.132829,414.269086,338.977538,337.581534,92.471705


In [10]:
pop_iris[pop_iris["iris"]==debug].to_excel(writer, sheet_name="iris", index=None)
pop_iris[pop_iris["iris"]==debug]


,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
38109,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916


In [108]:
ps_df = get_by_source(year, specialite, source)
ps_df

Quering PA for year 23 and specialite 21 for is_medecin=False

select pa.id, pa.inpp, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=21
--and ps.code_mode_exercice='L'
and ds.annee=23
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id, ps.code_mode_exercice



,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice
0,3975,810006157951,VEISS,CLAIRE,57,1426304,574630901,6.191237,49.111603,S
1,3976,810006158322,GARITEY,CHRISTINE,83,1264224,831370402,5.932466,43.131440,S
2,3977,810006162373,LOUIS DELAURIERE,EMILIE,75,1158435,751072604,2.319265,48.859561,S
3,3978,810006163215,MAROLLEAU,BRICE,13,1304266,132130303,5.413910,43.328556,S
4,3979,810006163850,PHILIPPE,PEGGY,57,1426304,574630901,6.191237,49.111603,S
...,...,...,...,...,...,...,...,...,...,...
437573,2541778,810108701664,DURAND,AMANDINE,71,156882,715200000,4.830783,46.726436,S
437574,2541779,810108716613,DHETZ,CAROLINE,73,158024,730870101,5.894233,45.563276,S
437575,2541782,810108722967,MOREAU,AURORE,23,1179942,231930000,1.894397,46.174576,S
437576,2541784,810108735126,LESCOT,MINDY,75,1246012,751155814,2.315430,48.846348,S


In [65]:
ps_df[ps_df["iris"]==debug].to_excel(writer, sheet_name="ps", index=None)
ps_df[ps_df["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice
11874,112683,810003138517,TYGA,FRANK,6,1581018,61210000,7.329490,43.691660,L
120529,1523827,810003278644,BAUDUIN,JACQUES,6,1460,61210000,7.333105,43.690969,L


In [109]:
nb_ps = ps_df["id"].nunique()
nb_cabinet_ps = ps_df.groupby(["id", "lon", "lat"])
print(f"Nb unique PS {nb_ps}")
print(f"Nb cabinet {len(nb_cabinet_ps)}")

Nb unique PS 382876
Nb cabinet 412632


In [110]:
ps_df["key"] = ps_df["id"].astype(str) + "_" + ps_df["code_mode_exercice"] + "_" + ps_df["lat"].astype(str) + "_" + ps_df["lon"].astype(str)
ps_df["nb_cabinet"] = ps_df.groupby("id")["key"].transform("nunique")
ps_df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1     0.811051
4     0.104194
2     0.071624
6     0.006088
3     0.005834
5     0.000503
8     0.000384
10    0.000137
12    0.000055
9     0.000041
7     0.000032
14    0.000032
11    0.000025
Name: proportion, dtype: float64

In [68]:
ps_df[ps_df["iris"]==debug].to_excel(writer, sheet_name="nb_cabinet", index=None)
ps_df[ps_df["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet
11874,112683,810003138517,TYGA,FRANK,6,1581018,61210000,7.329490,43.691660,L,112683_L_43.69166_7.32949,2
120529,1523827,810003278644,BAUDUIN,JACQUES,6,1460,61210000,7.333105,43.690969,L,1523827_L_43.690969_7.333105,1


In [111]:
ps_df = ps_df[ps_df["code_mode_exercice"]=="L"]
ps_df["weight"] = 1 / ps_df["nb_cabinet"]
ps_df["nb"] = ps_df.groupby("iris")["weight"].transform("sum")
ps_df.head(10)
# 0.5 2.17
# 0.5 8.83

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
22,5050,810102347498,BENAROUS,FRANCOISE,6,1213487,60110000,7.326919,43.702660,L,5050_L_43.70266_7.326919,1,1.0,13.250000
23,5051,810102349122,BUTTAFOGHI,PASCALE,6,1216641,60882504,7.253609,43.699982,L,5051_L_43.699982_7.253609,1,1.0,3.000000
24,5053,810102352316,MOUROU,MARJORIE,13,1256753,132150202,5.364120,43.367821,L,5053_L_43.367821_5.36412,1,1.0,3.000000
25,5055,810102354809,JOURDAIN,ANNE FRANCOISE,13,1178746,130010403,5.469916,43.537548,L,5055_L_43.537548_5.469916,1,1.0,6.500000
26,5056,810102359972,HUYBRECHTS,VERONIQUE,2,832188,26800000,3.377861,49.596710,L,5056_L_49.59671_3.377861,1,1.0,8.500000
27,5057,810102361176,CHABANNE,FRANCOISE,11,1171975,110690202,2.357005,43.211673,L,5057_L_43.211673_2.357005,1,1.0,11.000000
28,5058,810102362596,ARNAUD,MARIE PIERRE,13,404421,131030115,5.049572,43.642665,L,5058_L_43.642665_5.049572,1,1.0,13.000000
29,5059,810102364816,MOMPEU,MARIE THERESE,202,1206719,2020340000,9.007790,42.621887,L,5059_L_42.621887_9.00779,1,1.0,4.500000
30,5060,810102368569,GRAAS,MELANIE,13,1174133,690290401,4.911930,45.733312,L,5060_L_45.733312_4.91193,1,1.0,13.916667
31,5061,810102368874,MAZOUARD,BRIGITTE,26,1185532,260450000,4.920357,44.546287,L,5061_L_44.546287_4.920357,1,1.0,4.166667


In [70]:
ps_df[ps_df["iris"]==debug].to_excel(writer, sheet_name="nb", index=None)
ps_df[ps_df["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
11874,112683,810003138517,TYGA,FRANK,6,1581018,61210000,7.329490,43.691660,L,112683_L_43.69166_7.32949,2,0.5,1.5
120529,1523827,810003278644,BAUDUIN,JACQUES,6,1460,61210000,7.333105,43.690969,L,1523827_L_43.690969_7.333105,1,1.0,1.5


In [112]:
ps_df = ps_df.sort_values(by='iris')
ps_df2 = ps_df.drop_duplicates(subset=['iris', 'nb'])
ps_df2


,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
309298,1251885,810102584116,VUILLON,CELINE,1,1254439,10040101,5.344378,45.962699,L,1251885_L_45.962699_5.344378,1,1.00,6.250000
12538,53335,810102985131,NASSI,SELIM,1,1165864,10040102,5.333877,45.955981,L,53335_L_45.955981_5.333877,4,0.25,8.500000
307645,1241932,810106267387,MOISAN,ANNE MARIE,1,1238031,10040201,5.358693,45.958089,L,1241932_L_45.958089_5.358693,1,1.00,9.916667
226887,903338,810106329542,CAROLLO,CELINE,1,342978,10040202,5.365760,45.949967,L,903338_L_45.949967_5.36576,4,0.25,3.250000
113070,448979,810102454161,ETIENNE,MARIE PIERRE,1,685948,10050000,4.904387,45.995929,L,448979_L_45.995929_4.904387,1,1.00,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219587,873542,810106057499,GLAVIEUX RODRIGUES,SAMUEL,202,1210583,2023410000,9.178979,42.233594,L,873542_L_42.233594_9.178979,2,0.50,2.500000
8835,39318,810102821518,ZERBINI,CLEMENTINE,202,575234,2023420000,9.380487,41.965795,L,39318_L_41.965795_9.380487,4,0.25,6.250000
151328,599869,810102843421,FILIPPI,CELIA,202,1258826,2023460000,9.450628,42.507081,L,599869_L_42.507081_9.450628,4,0.25,2.250000
163605,648999,810102580163,FAZI,MARIE CATHERINE,202,604014,2023470000,9.245658,42.173746,L,648999_L_42.173746_9.245658,1,1.00,2.000000


In [72]:
ps_df2[ps_df2["iris"]==debug].to_excel(writer, sheet_name="drop", index=None)
ps_df2[ps_df2["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
120529,1523827,810003278644,BAUDUIN,JACQUES,6,1460,61210000,7.333105,43.690969,L,1523827_L_43.690969_7.333105,1,1.0,1.5


In [113]:
iris_matrix_pop_df = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', '')) #iris = iris2
iris_matrix_pop_df

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.0,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.0,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.0,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.0,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000,2B1350000,2B135,C,324.0,3.980971,2.049514,2.049514,2.020847,18.774109,42.392738,42.077099,55.572529,98.952018,56.130660
19272671,2023650000,2022510000,10,15,15,15,2022510000,2B2510000,2B251,C,3737.0,107.555612,118.720070,241.423529,317.512086,210.704067,631.501489,774.648158,501.913516,523.962099,309.059373
19272672,2023650000,2022770000,16,28,28,28,2022770000,2B2770000,2B277,C,333.0,4.577269,2.653059,12.288707,23.776840,23.014588,32.395245,76.114846,64.660108,67.979158,25.540181
19272673,2023650000,2023650000,0,0,0,0,2023650000,2B3650000,2B365,C,100.0,0.000000,0.943396,0.000000,3.773585,9.433962,9.433962,9.433962,23.584906,29.245283,14.150943


In [114]:
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug].to_excel(writer, sheet_name="iris_matrix", index=None)
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
273369,60110000,61210000,3,5,6,5,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
273582,60120101,61210000,16,24,32,24,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
273781,60120102,61210000,15,23,30,23,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
273985,60120103,61210000,13,21,28,21,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
274194,60120104,61210000,14,22,29,22,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377769,61590103,61210000,7,12,17,12,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
378455,61610102,61210000,24,30,40,30,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
378812,61610103,61210000,23,29,38,29,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916
379174,61610104,61210000,22,30,40,30,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916


In [115]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)
iris_matrix_pop_df["accessibilite_weight"] = accessibilite_fn(iris_matrix_pop_df["time"])
iris_matrix_pop_df.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.000000,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.000000,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108,0.056135
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.000000,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055,0.027324
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.000000,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300,0.034735
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.000000,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472,0.049787
5,10010000,10280000,8,9,9,9,10280000,010280000,01028,C,621.000000,22.723364,29.574898,53.956390,70.586718,22.773615,107.898596,115.308426,89.365644,83.584630,25.227720,0.339596
6,10010000,10420000,14,16,16,16,10420000,010420000,01042,C,290.000000,3.133037,7.312245,22.902721,37.112788,10.199399,41.375370,72.152604,45.323311,42.956047,7.532476,0.146607
7,10010000,10460000,12,13,13,13,10460000,010460000,01046,C,904.000000,36.261938,27.238169,71.537021,113.956388,45.490399,158.230805,197.653434,122.489017,101.564554,29.578275,0.210136
8,10010000,10520000,17,21,22,21,10520000,010520000,01052,C,327.000000,8.933497,12.331319,22.701646,25.430244,8.343897,43.396597,63.486997,59.830525,57.972826,24.572452,0.080460
9,10010000,10530102,26,28,34,28,10530102,010530102,01053,A,1925.526865,47.250779,43.545566,63.504863,104.972858,183.815536,325.024244,323.908369,162.260767,385.334404,285.909479,0.034735


In [116]:
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug].to_excel(writer, sheet_name="accessibilite", index=None)
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
273369,60110000,61210000,3,5,6,5,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.548812
273582,60120101,61210000,16,24,32,24,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.056135
273781,60120102,61210000,15,23,30,23,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.063292
273985,60120103,61210000,13,21,28,21,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.080460
274194,60120104,61210000,14,22,29,22,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.071361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377769,61590103,61210000,7,12,17,12,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.236928
378455,61610102,61210000,24,30,40,30,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.027324
378812,61610103,61210000,23,29,38,29,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.030807
379174,61610104,61210000,22,30,40,30,61210000,061210000,06121,C,1471.0,26.88639,31.534309,55.567568,79.991458,83.865916,206.509731,307.039686,243.487553,300.013473,136.103916,0.027324


In [117]:
cols = [col for col in iris_matrix_pop_df.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [118]:
def get_over(year, specialite, is_medecin):
    yy=min(year, 24)
    if is_medecin:
        sql = f"""select o.* from apl.overrepresentation o
        join specialite s on s.psp_spe_snds=o.psp_spe_snds
        where o.year={yy}
        and s.id={specialite}
        """
    else:
        sql = f"""select o.* from apl.overrepresentation o
        join specialite s on s.psp_act_snds=o.psp_act_snds
        where o.year={yy}
        and s.id={specialite}
        """
    over = pd.read_sql(sql, config.connection_string)
    return over

over = get_over(year, specialite, specialite<21)
over

,year,psp_spe_snds,psp_act_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,23,None,24.0,0.239073,0.22113,0.206107,0.20108,0.375333,0.503999,0.734527,1.16381,2.070894,4.028416


In [79]:
over.to_excel(writer, sheet_name="over", index=None)

In [119]:
if len(over) > 0:
    weights = over.values[0,3:]
else:
    weights = np.ones(len(over.columns) - 3)
weights

array([0.23907322106450268, 0.22113007374488441, 0.20610710896092563,
       0.20108049241941214, 0.37533299477824317, 0.5039990020580614,
       0.7345273330087865, 1.163809741174566, 2.0708937918811614,
       4.028415814155608], dtype=object)

In [120]:
iris_matrix_pop_df["pop_gp"] = sum(w * iris_matrix_pop_df[c] for w, c in zip(weights, cols))
if specialite == 5: # todo ajouter sage femme
    iris_matrix_pop_df["pop_gp"] /= 2
iris_matrix_pop_df=iris_matrix_pop_df.sort_values(by='iris2')
iris_matrix_pop_df.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000,822.550993
6351493,712700102,10010000,25,27,33,27,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.039164,822.550993
6351706,712700103,10010000,24,26,32,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,822.550993
60021,14430000,10010000,21,26,27,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,822.550993
6351935,712700104,10010000,25,28,34,28,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.034735,822.550993


In [121]:
ratio_mean=1
if specialite!=5:
    test_pop=iris_matrix_pop_df.drop_duplicates(subset=['iris2'])
    ratio=test_pop["pop_gp"]/(test_pop["pop"]+1)
    ratio_mean=np.mean(ratio)
    print(ratio_mean)
    iris_matrix_pop_df["pop_gp"]=iris_matrix_pop_df["pop_gp"]/ratio_mean

1.040278076070029


In [122]:
matrix_df = iris_matrix_pop_df[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
matrix_df.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,10010000,10010000,0,0,1.000000,790.702997,832.0,C
6351493,712700102,10010000,25,27,0.039164,790.702997,832.0,C
6351706,712700103,10010000,24,26,0.044157,790.702997,832.0,C
60021,14430000,10010000,21,26,0.044157,790.702997,832.0,C
6351935,712700104,10010000,25,28,0.034735,790.702997,832.0,C


In [123]:
matrix_df[matrix_df["iris2"]==debug].to_excel(writer, sheet_name=f"pop_ajustee_{ratio_mean:.3f}", index=None)
matrix_df[matrix_df["iris2"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
352810,60882805,61210000,15,22,0.071361,1783.156268,1471.0,C
338207,60881703,61210000,9,14,0.186374,1783.156268,1471.0,C
314685,60880203,61210000,7,12,0.236928,1783.156268,1471.0,C
322580,60880605,61210000,9,15,0.165299,1783.156268,1471.0,C
323697,60880901,61210000,11,17,0.130029,1783.156268,1471.0,C
...,...,...,...,...,...,...,...,...
346128,60882304,61210000,9,15,0.165299,1783.156268,1471.0,C
377305,61590101,61210000,4,7,0.431711,1783.156268,1471.0,C
281685,60270116,61210000,18,25,0.049787,1783.156268,1471.0,C
321342,60880601,61210000,8,13,0.210136,1783.156268,1471.0,C


In [124]:
matrix_df["iris"] = matrix_df["iris1"].copy()
matrix_df = matrix_df.sort_values(by='iris2')
matrix_merge_df = matrix_df.merge(ps_df2, on="iris", how="left", suffixes=('', ''))
matrix_merge_df

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
0,10010000,10010000,0,0,1.000000,790.702997,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10280000,10010000,8,9,0.339596,790.702997,832.0,C,10280000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12890101,10010000,27,27,0.039164,790.702997,832.0,C,12890101,1340435.0,810102374583,MAITREPIERRE,LAURENCE,1.0,272743.0,5.211489,46.178155,L,1340435_L_46.178155_5.211489,1.0,1.00,9.00
3,12890102,10010000,28,28,0.034735,790.702997,832.0,C,12890102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12910000,10010000,18,23,0.063292,790.702997,832.0,C,12910000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2023650000,0,0,1.000000,154.968754,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272671,2021350000,2023650000,6,8,0.382893,154.968754,100.0,C,2021350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272672,2022510000,2023650000,10,15,0.165299,154.968754,100.0,C,2022510000,487195.0,810102762811,HENRI,SEVERINE,202.0,1210704.0,9.394775,42.001905,L,487195_L_42.001905_9.394775,1.0,1.00,21.75
19272673,2021230000,2023650000,17,25,0.049787,154.968754,100.0,C,2021230000,1541425.0,810102387668,SANCHEZ,NATHALIE,202.0,1159573.0,9.405117,42.014953,L,1541425_L_42.014953_9.405117,4.0,0.25,14.75


In [125]:
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)].to_excel(writer, sheet_name=f"join_iris", index=None)
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)]


,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
265999,61210000,60040104,29,30,0.027324,2424.918160,2163.470849,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274254,61210000,60110000,3,5,0.548812,4427.976195,3796.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274420,61210000,60120101,16,25,0.049787,2269.038091,2024.368949,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274701,61210000,60120102,15,23,0.063292,2408.755088,2580.015848,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274880,61210000,60120103,13,21,0.080460,2169.141131,2377.875116,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379511,61210000,61610102,25,30,0.027324,1840.324899,1986.426435,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
379867,61210000,61610103,24,29,0.030807,1952.158390,1938.158148,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
380243,61210000,61610104,22,30,0.027324,2191.731592,2096.158881,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
380613,61210000,61610105,25,28,0.034735,4125.943504,3584.453834,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0


In [126]:
matrix_merge_df["nb"] = matrix_merge_df["nb"].fillna(0)
matrix_merge_df = matrix_merge_df.sort_values(by=['iris1', "iris2"])
matrix_merge_df["nb"].nunique()

461

In [127]:
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)].to_excel(writer, sheet_name=f"nb", index=None)
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
265999,61210000,60040104,29,30,0.027324,2424.918160,2163.470849,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274254,61210000,60110000,3,5,0.548812,4427.976195,3796.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274420,61210000,60120101,16,25,0.049787,2269.038091,2024.368949,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274701,61210000,60120102,15,23,0.063292,2408.755088,2580.015848,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
274880,61210000,60120103,13,21,0.080460,2169.141131,2377.875116,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379511,61210000,61610102,25,30,0.027324,1840.324899,1986.426435,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
379867,61210000,61610103,24,29,0.030807,1952.158390,1938.158148,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
380243,61210000,61610104,22,30,0.027324,2191.731592,2096.158881,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0
380613,61210000,61610105,25,28,0.034735,4125.943504,3584.453834,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0


In [128]:
# matrix_merge_df = matrix_merge_df.sort_values(by='iris1')
matrix_merge_df["wpop"] = matrix_merge_df["accessibilite_weight"] * matrix_merge_df["pop_gp"]
matrix_merge_df["swpop"] = matrix_merge_df.groupby("iris1")["wpop"].transform("sum")
matrix_merge_df["R"] = matrix_merge_df["nb"] / (matrix_merge_df["swpop"] / 100000)
matrix_merge_df.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R
0,10010000,10010000,0,0,1.000000,790.702997,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,790.702997,19133.494274,0.0
917,10010000,10050000,21,24,0.056135,1668.574147,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,93.665014,19133.494274,0.0
2030,10010000,10210000,25,30,0.027324,1277.298850,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,34.900559,19133.494274,0.0
2594,10010000,10250000,22,28,0.034735,3507.473382,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,121.832996,19133.494274,0.0
2739,10010000,10260000,20,25,0.049787,1089.884917,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.262175,19133.494274,0.0


In [129]:
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)].to_excel(writer, sheet_name=f"R", index=None)
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R
265999,61210000,60040104,29,30,0.027324,2424.918160,2163.470849,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,66.257791,66785.087411,5.989361
274254,61210000,60110000,3,5,0.548812,4427.976195,3796.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,2430.124860,66785.087411,5.989361
274420,61210000,60120101,16,25,0.049787,2269.038091,2024.368949,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,112.968755,66785.087411,5.989361
274701,61210000,60120102,15,23,0.063292,2408.755088,2580.015848,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,152.454369,66785.087411,5.989361
274880,61210000,60120103,13,21,0.080460,2169.141131,2377.875116,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,174.528242,66785.087411,5.989361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379511,61210000,61610102,25,30,0.027324,1840.324899,1986.426435,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,50.284527,66785.087411,5.989361
379867,61210000,61610103,24,29,0.030807,1952.158390,1938.158148,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,60.140946,66785.087411,5.989361
380243,61210000,61610104,22,30,0.027324,2191.731592,2096.158881,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,59.886266,66785.087411,5.989361
380613,61210000,61610105,25,28,0.034735,4125.943504,3584.453834,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,143.315716,66785.087411,5.989361


In [130]:
rgp = matrix_merge_df[matrix_merge_df["iris1"] == matrix_merge_df["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
0,10010000,10010000,C,790.702997,0.000000,0.000000,19133.494274,790.702997,790.702997,832.000000
177,10020000,10020000,C,242.006732,0.000000,0.000000,8602.799088,242.006732,242.006732,267.000000
265,10040101,10040101,A,1524.501903,6.250000,20.236235,30885.192594,1524.501903,1524.501903,1796.951774
470,10040102,10040102,A,3431.019785,8.500000,29.470940,28841.971324,3431.019785,3431.019785,3837.421806
585,10040201,10040201,A,4126.386485,9.916667,34.161420,29028.847779,4126.386485,4126.386485,4393.228271
...,...,...,...,...,...,...,...,...,...,...
19272636,2023560000,2023560000,C,232.730248,0.000000,0.000000,284.294690,232.730248,232.730248,148.000000
19272651,2023610000,2023610000,C,319.334308,0.000000,0.000000,1913.852047,319.334308,319.334308,306.000000
19272667,2023640000,2023640000,C,61.443054,0.000000,0.000000,188.084826,61.443054,61.443054,36.000000
19272670,2023650000,2023650000,C,154.968754,0.000000,0.000000,1225.315417,154.968754,154.968754,100.000000


In [131]:
rgp[(rgp["iris1"]==debug) | (rgp["iris1"]==debug)].to_excel(writer, sheet_name=f"rgp", index=None)
rgp[(rgp["iris1"]==debug) | (rgp["iris1"]==debug)]

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
365484,61210000,61210000,C,1783.156268,4.0,5.989361,66785.087411,1783.156268,1783.156268,1471.0


In [132]:
apl = matrix_merge_df.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,10010000,10010000,0,0,1.000000,790.702997,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,790.702997,19133.494274,0.000000,10010000,C,790.702997,0.0,0.0,19133.494274,790.702997,790.702997,832.0
1,10050000,10010000,21,24,0.056135,790.702997,832.0,C,10050000,448979.0,810102454161,ETIENNE,MARIE PIERRE,1.0,685948.0,4.904387,45.995929,L,448979_L_45.995929_4.904387,1.0,1.0,3.0,44.385925,36162.327124,8.295926,10010000,C,790.702997,0.0,0.0,19133.494274,790.702997,790.702997,832.0
2,10210000,10010000,25,30,0.027324,790.702997,832.0,C,10210000,1424015.0,810106507592,PAGET,MELANIE,1.0,1212778.0,4.823181,45.992168,L,1424015_L_45.992168_4.823181,1.0,1.0,3.0,21.604949,44176.881541,6.790882,10010000,C,790.702997,0.0,0.0,19133.494274,790.702997,790.702997,832.0
3,10250000,10010000,22,28,0.034735,790.702997,832.0,C,10250000,494769.0,810106311102,BUCHET,CELINE,1.0,1227087.0,4.945669,46.316032,L,494769_L_46.316032_4.945669,1.0,1.0,3.0,27.465273,25788.558460,11.633066,10010000,C,790.702997,0.0,0.0,19133.494274,790.702997,790.702997,832.0
4,10260000,10010000,20,25,0.049787,790.702997,832.0,C,10260000,1223830.0,810102374948,PERRIER,MAGALI,1.0,644930.0,4.928212,46.312266,L,1223830_L_46.312266_4.928212,1.0,1.0,3.5,39.366784,33821.230075,10.348530,10010000,C,790.702997,0.0,0.0,19133.494274,790.702997,790.702997,832.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2022730000,2022970000,3,7,0.431711,56.772253,53.0,C,2022730000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,24.509179,366.798559,0.000000,2022970000,C,56.772253,0.0,0.0,158.759199,56.772253,56.772253,53.0
19272671,2022970000,2022970000,0,0,1.000000,56.772253,53.0,C,2022970000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,56.772253,158.759199,0.000000,2022970000,C,56.772253,0.0,0.0,158.759199,56.772253,56.772253,53.0
19272672,2022990000,2022970000,6,11,0.267135,56.772253,53.0,C,2022990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.165873,725.773837,0.000000,2022970000,C,56.772253,0.0,0.0,158.759199,56.772253,56.772253,53.0
19272673,2023440000,2022970000,11,23,0.063292,56.772253,53.0,C,2023440000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.593216,684.111646,0.000000,2022970000,C,56.772253,0.0,0.0,158.759199,56.772253,56.772253,53.0


In [133]:
apl[apl["iris"]==debug].to_excel(writer, sheet_name=f"join_iris1", index=None)
apl[apl["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
1042080,61210000,60040104,29,30,0.027324,2424.918160,2163.470849,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,66.257791,66785.087411,5.989361,60040104,A,2424.918160,0.000000,0.000000,181931.207894,2424.918160,2424.918160,2163.470849
1051238,61210000,60270101,21,29,0.030807,5091.024217,4687.508879,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,156.841276,66785.087411,5.989361,60270101,A,5091.024217,51.000000,26.619291,191590.378211,5091.024217,5091.024217,4687.508879
1051603,61210000,60270102,23,27,0.039164,3497.203189,4540.461241,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,136.964099,66785.087411,5.989361,60270102,A,3497.203189,13.000000,6.733001,193078.846212,3497.203189,3497.203189,4540.461241
1051971,61210000,60270103,22,27,0.039164,3120.869658,2761.785324,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,122.225412,66785.087411,5.989361,60270103,A,3120.869658,2.166667,1.091482,198506.899139,3120.869658,3120.869658,2761.785324
1052330,61210000,60270104,24,28,0.034735,3620.018902,3353.315676,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,125.742294,66785.087411,5.989361,60270104,A,3620.018902,1.500000,0.910223,164794.863424,3620.018902,3620.018902,3353.315676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152833,61210000,61490104,14,24,0.056135,2988.311279,3217.369805,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,167.748145,66785.087411,5.989361,61490104,A,2988.311279,1.000000,1.172334,85299.923973,2988.311279,2988.311279,3217.369805
1153178,61210000,60110000,3,5,0.548812,4427.976195,3796.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,2430.124860,66785.087411,5.989361,60110000,C,4427.976195,13.250000,17.387468,76204.307940,4427.976195,4427.976195,3796.000000
1153557,61210000,60590000,11,15,0.165299,2448.718794,2189.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,404.770494,66785.087411,5.989361,60590000,C,2448.718794,0.000000,0.000000,78969.161588,2448.718794,2448.718794,2189.000000
1154376,61210000,61040102,20,29,0.030807,3493.508932,3061.125143,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,107.625966,66785.087411,5.989361,61040102,A,3493.508932,5.000000,12.392163,40348.080893,3493.508932,3493.508932,3061.125143


In [134]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap
0,10010000,10010000,0,0,1.000000,790.702997,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,790.702997,19133.494274,0.0,10010000,C,790.702997,0.0,0.000000,19133.494274,790.702997,790.702997,832.0,0.000000
155,10010000,10050000,21,24,0.056135,1668.574147,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,93.665014,19133.494274,0.0,10050000,C,1668.574147,3.0,8.295926,36162.327124,1668.574147,1668.574147,1897.0,0.465690
386,10010000,10210000,25,30,0.027324,1277.298850,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,34.900559,19133.494274,0.0,10210000,C,1277.298850,3.0,6.790882,44176.881541,1277.298850,1277.298850,1498.0,0.185552
653,10010000,10250000,22,28,0.034735,3507.473382,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,121.832996,19133.494274,0.0,10250000,C,3507.473382,3.0,11.633066,25788.558460,3507.473382,3507.473382,4065.0,0.404078
808,10010000,10260000,20,25,0.049787,1089.884917,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.262175,19133.494274,0.0,10260000,C,1089.884917,3.5,10.348530,33821.230075,1089.884917,1089.884917,979.0,0.515223


In [135]:
apl[apl["iris"]==debug].to_excel(writer, sheet_name=f"ap", index=None)
apl[apl["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap
1042080,61210000,60040104,29,30,0.027324,2424.918160,2163.470849,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,66.257791,66785.087411,5.989361,60040104,A,2424.918160,0.000,0.000000,181931.207894,2424.918160,2424.918160,2163.470849,0.000000
1153178,61210000,60110000,3,5,0.548812,4427.976195,3796.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,2430.124860,66785.087411,5.989361,60110000,C,4427.976195,13.250,17.387468,76204.307940,4427.976195,4427.976195,3796.000000,9.542445
1150010,61210000,60120101,16,25,0.049787,2269.038091,2024.368949,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,112.968755,66785.087411,5.989361,60120101,A,2269.038091,0.250,0.414655,60291.013248,2269.038091,2269.038091,2024.368949,0.020644
1150216,61210000,60120102,15,23,0.063292,2408.755088,2580.015848,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,152.454369,66785.087411,5.989361,60120102,A,2408.755088,4.000,8.107922,49334.468675,2408.755088,2408.755088,2580.015848,0.513165
1150422,61210000,60120103,13,21,0.080460,2169.141131,2377.875116,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,174.528242,66785.087411,5.989361,60120103,A,2169.141131,2.250,3.655653,61548.510018,2169.141131,2169.141131,2377.875116,0.294132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146370,61210000,61610102,25,30,0.027324,1840.324899,1986.426435,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,50.284527,66785.087411,5.989361,61610102,A,1840.324899,7.000,4.926585,142086.265585,1840.324899,1840.324899,1986.426435,0.134613
1146727,61210000,61610103,24,29,0.030807,1952.158390,1938.158148,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,60.140946,66785.087411,5.989361,61610103,A,1952.158390,2.500,1.513273,165204.815719,1952.158390,1952.158390,1938.158148,0.046620
1147084,61210000,61610104,22,30,0.027324,2191.731592,2096.158881,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,59.886266,66785.087411,5.989361,61610104,A,2191.731592,3.000,1.576641,190277.936621,2191.731592,2191.731592,2096.158881,0.043080
1147446,61210000,61610105,25,28,0.034735,4125.943504,3584.453834,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,143.315716,66785.087411,5.989361,61610105,A,4125.943504,4.625,2.231910,207221.594154,4125.943504,4125.943504,3584.453834,0.077526


In [136]:
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.000000,790.702997,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,790.702997,19133.494274,0.0,10010000,C,790.702997,0.00,0.000000,19133.494274,790.702997,790.702997,832.0,0.000000,99.511050
155,10010000,10050000,21,24,0.056135,1668.574147,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,93.665014,19133.494274,0.0,10050000,C,1668.574147,3.00,8.295926,36162.327124,1668.574147,1668.574147,1897.0,0.465690,99.511050
386,10010000,10210000,25,30,0.027324,1277.298850,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,34.900559,19133.494274,0.0,10210000,C,1277.298850,3.00,6.790882,44176.881541,1277.298850,1277.298850,1498.0,0.185552,99.511050
653,10010000,10250000,22,28,0.034735,3507.473382,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,121.832996,19133.494274,0.0,10250000,C,3507.473382,3.00,11.633066,25788.558460,3507.473382,3507.473382,4065.0,0.404078,99.511050
808,10010000,10260000,20,25,0.049787,1089.884917,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.262175,19133.494274,0.0,10260000,C,1089.884917,3.50,10.348530,33821.230075,1089.884917,1089.884917,979.0,0.515223,99.511050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272636,2023650000,2021350000,6,8,0.382893,535.689032,324.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,205.111519,1225.315417,0.0,2021350000,C,535.689032,0.00,0.000000,1915.879383,535.689032,535.689032,324.0,0.000000,80.248989
19272607,2023650000,2022510000,10,15,0.165299,3889.491520,3737.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,642.928624,1225.315417,0.0,2022510000,C,3889.491520,21.75,398.180766,5462.343208,3889.491520,3889.491520,3737.0,65.818838,80.248989
19272612,2023650000,2022770000,16,28,0.034735,392.957153,333.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.649468,1225.315417,0.0,2022770000,C,392.957153,1.00,68.477422,1460.335357,392.957153,392.957153,333.0,2.378581,80.248989
19272620,2023650000,2023650000,0,0,1.000000,154.968754,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,154.968754,1225.315417,0.0,2023650000,C,154.968754,0.00,0.000000,1225.315417,154.968754,154.968754,100.0,0.000000,80.248989


In [137]:
apl[apl["iris"]==debug].to_excel(writer, sheet_name=f"sum_ap", index=None)
apl[apl["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
1042080,61210000,60040104,29,30,0.027324,2424.918160,2163.470849,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,66.257791,66785.087411,5.989361,60040104,A,2424.918160,0.000,0.000000,181931.207894,2424.918160,2424.918160,2163.470849,0.000000,99.3811
1153178,61210000,60110000,3,5,0.548812,4427.976195,3796.000000,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,2430.124860,66785.087411,5.989361,60110000,C,4427.976195,13.250,17.387468,76204.307940,4427.976195,4427.976195,3796.000000,9.542445,99.3811
1150010,61210000,60120101,16,25,0.049787,2269.038091,2024.368949,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,112.968755,66785.087411,5.989361,60120101,A,2269.038091,0.250,0.414655,60291.013248,2269.038091,2269.038091,2024.368949,0.020644,99.3811
1150216,61210000,60120102,15,23,0.063292,2408.755088,2580.015848,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,152.454369,66785.087411,5.989361,60120102,A,2408.755088,4.000,8.107922,49334.468675,2408.755088,2408.755088,2580.015848,0.513165,99.3811
1150422,61210000,60120103,13,21,0.080460,2169.141131,2377.875116,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,174.528242,66785.087411,5.989361,60120103,A,2169.141131,2.250,3.655653,61548.510018,2169.141131,2169.141131,2377.875116,0.294132,99.3811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146370,61210000,61610102,25,30,0.027324,1840.324899,1986.426435,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,50.284527,66785.087411,5.989361,61610102,A,1840.324899,7.000,4.926585,142086.265585,1840.324899,1840.324899,1986.426435,0.134613,99.3811
1146727,61210000,61610103,24,29,0.030807,1952.158390,1938.158148,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,60.140946,66785.087411,5.989361,61610103,A,1952.158390,2.500,1.513273,165204.815719,1952.158390,1952.158390,1938.158148,0.046620,99.3811
1147084,61210000,61610104,22,30,0.027324,2191.731592,2096.158881,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,59.886266,66785.087411,5.989361,61610104,A,2191.731592,3.000,1.576641,190277.936621,2191.731592,2191.731592,2096.158881,0.043080,99.3811
1147446,61210000,61610105,25,28,0.034735,4125.943504,3584.453834,A,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,143.315716,66785.087411,5.989361,61610105,A,4125.943504,4.625,2.231910,207221.594154,4125.943504,4125.943504,3584.453834,0.077526,99.3811


In [138]:
apl2 = apl[apl["iris1"] == apl["iris2"]]
apl2

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.0,790.702997,832.000000,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,790.702997,19133.494274,0.000000,10010000,C,790.702997,0.000000,0.000000,19133.494274,790.702997,790.702997,832.000000,0.000000,99.511050
28748,10020000,10020000,0,0,1.0,242.006732,267.000000,C,10020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,242.006732,8602.799088,0.000000,10020000,C,242.006732,0.000000,0.000000,8602.799088,242.006732,242.006732,267.000000,0.000000,82.294189
28815,10040101,10040101,0,0,1.0,1524.501903,1796.951774,A,10040101,1251885.0,810102584116,VUILLON,CELINE,1.0,1254439.0,5.344378,45.962699,L,1251885_L_45.962699_5.344378,1.0,1.00,6.250000,1524.501903,30885.192594,20.236235,10040101,A,1524.501903,6.250000,20.236235,30885.192594,1524.501903,1524.501903,1796.951774,20.236235,237.132735
28999,10040102,10040102,0,0,1.0,3431.019785,3837.421806,A,10040102,53335.0,810102985131,NASSI,SELIM,1.0,1165864.0,5.333877,45.955981,L,53335_L_45.955981_5.333877,4.0,0.25,8.500000,3431.019785,28841.971324,29.470940,10040102,A,3431.019785,8.500000,29.470940,28841.971324,3431.019785,3431.019785,3837.421806,29.470940,233.840987
29143,10040201,10040201,0,0,1.0,4126.386485,4393.228271,A,10040201,1241932.0,810106267387,MOISAN,ANNE MARIE,1.0,1238031.0,5.358693,45.958089,L,1241932_L_45.958089_5.358693,1.0,1.00,9.916667,4126.386485,29028.847779,34.161420,10040201,A,4126.386485,9.916667,34.161420,29028.847779,4126.386485,4126.386485,4393.228271,34.161420,229.339553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19268570,2023560000,2023560000,0,0,1.0,232.730248,148.000000,C,2023560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,232.730248,284.294690,0.000000,2023560000,C,232.730248,0.000000,0.000000,284.294690,232.730248,232.730248,148.000000,0.000000,0.000000
19268441,2023610000,2023610000,0,0,1.0,319.334308,306.000000,C,2023610000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,319.334308,1913.852047,0.000000,2023610000,C,319.334308,0.000000,0.000000,1913.852047,319.334308,319.334308,306.000000,0.000000,81.856087
19268523,2023640000,2023640000,0,0,1.0,61.443054,36.000000,C,2023640000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,61.443054,188.084826,0.000000,2023640000,C,61.443054,0.000000,0.000000,188.084826,61.443054,61.443054,36.000000,0.000000,0.000000
19272620,2023650000,2023650000,0,0,1.0,154.968754,100.000000,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,154.968754,1225.315417,0.000000,2023650000,C,154.968754,0.000000,0.000000,1225.315417,154.968754,154.968754,100.000000,0.000000,80.248989


In [139]:
apl2[apl2["iris"]==debug].to_excel(writer, sheet_name=f"apl", index=None)
writer.close()
apl2[apl2["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
1154718,61210000,61210000,0,0,1.0,1783.156268,1471.0,C,61210000,1230576.0,810105434525,GUILLOUX GRONDIN,NELLY,6.0,1207693.0,7.326574,43.700266,L,1230576_L_43.7002663_7.3265736,1.0,1.0,4.0,1783.156268,66785.087411,5.989361,61210000,C,1783.156268,4.0,5.989361,66785.087411,1783.156268,1783.156268,1471.0,5.989361,99.3811


In [140]:
print(year, specialite, source)
apl2["apl"].describe()
# 21-10-PA:67-57 avec ratio:68-58
# 20-10-PA:68-58 avec ratio:69-59

23 21 PA


count    48569.000000
mean       121.609668
std         68.201018
min          0.000000
25%         75.894831
50%        103.215284
75%        152.406303
max       2759.892519
Name: apl, dtype: float64

In [141]:
# 21 10 PA
# count    48569.000000
# mean        66.239008
# std         36.200523
# min          0.000000
# 25%         40.432111
# 50%         56.872725
# 75%         85.083597
# max        681.212229

# 23 10 PA
# count    48569.000000
# mean        67.704332
# std         37.706173
# min          0.000000
# 25%         40.971273
# 50%         57.986870
# 75%         86.747319
# max        671.098376

# Avec prise en compte des cabinets non L
# 23 10 PA
# count    48569.000000
# mean        64.104830
# std         35.929837
# min          0.000000
# 25%         38.716316
# 50%         55.025585
# 75%         82.063870
# max        671.098376

# 24 10 PA
# count    48569.000000
# mean        62.960575
# std         36.148895
# min          0.000000
# 25%         37.356832
# 50%         53.792301
# 75%         80.853780
# max        671.109886

# 25 10 PA
# count    48569.000000
# mean        63.364476
# std         36.585777
# min          0.000000
# 25%         37.419434
# 50%         53.987877
# 75%         81.566145
# max        671.109886

In [142]:
dico = {"year":year, "specialite_id":specialite, "source":source, "time":time, "time_type":time_type, "exp":accessibilite_exp}
dico["mean"] = np.mean(apl2["apl"])
dico["std"] = np.std(apl2["apl"])
dico["q10"], dico["q25"], dico["q50"], dico["q75"], dico["q90"] = np.quantile(apl2["apl"], [0.1, 0.25, 0.5, 0.75, 0.9])
dico["min"] = np.min(apl2["apl"])
dico["max"] = np.max(apl2["apl"])
dico["date"] = datetime.datetime.now()
dico["key"] = random.randint(0, 1000000000000)
dico

{'year': 23,
 'specialite_id': 21,
 'source': 'PA',
 'time': 30,
 'time_type': 'HC',
 'exp': -0.12,
 'mean': 121.60966819148804,
 'std': 68.20031625635494,
 'q10': 57.233813800859465,
 'q25': 75.89483137048173,
 'q50': 103.21528397818277,
 'q75': 152.40630267654427,
 'q90': 209.33804240941544,
 'min': 0.0,
 'max': 2759.8925193094547,
 'date': datetime.datetime(2025, 11, 18, 11, 30, 19, 301256),
 'key': 89349514367}

In [143]:
study = pd.DataFrame(dico, index=[dico["key"]])
study

,year,specialite_id,source,time,time_type,exp,mean,std,q10,q25,q50,q75,q90,min,max,date,key
89349514367,23,21,PA,30,HC,-0.12,121.609668,68.200316,57.233814,75.894831,103.215284,152.406303,209.338042,0.0,2759.892519,2025-11-18 11:30:19.301256,89349514367


In [144]:
apl3 = apl2[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl3 = apl3.rename(columns={'iris1': 'iris'})
apl3.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
0,10010000,C,0.000000,99.511050,0.000000,1.0,790.702997,19133.494274,0.000000,790.702997,832.000000
28748,10020000,C,0.000000,82.294189,0.000000,1.0,242.006732,8602.799088,0.000000,242.006732,267.000000
28815,10040101,A,6.250000,237.132735,20.236235,1.0,1524.501903,30885.192594,20.236235,1524.501903,1796.951774
28999,10040102,A,8.500000,233.840987,29.470940,1.0,3431.019785,28841.971324,29.470940,3431.019785,3837.421806
29143,10040201,A,9.916667,229.339553,34.161420,1.0,4126.386485,29028.847779,34.161420,4126.386485,4393.228271


In [145]:
apl3[apl3["iris"]==61070000]

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
1036622,61070000,C,0.0,153.359514,0.0,1.0,58.456659,893.24945,0.0,58.456659,65.0


In [146]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2023 "year", 21 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id



,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,821300000,821300000,Montjoi,82,82130,Montjoi,2023,21
1,811540000,811540000,Marnaves,81,81154,Marnaves,2023,21
2,840060000,840060000,Auribeau,84,84006,Auribeau,2023,21
3,821570000,821570000,Saint-Beauzeil,82,82157,Saint-Beauzeil,2023,21
4,840070113,840070113,Carmes-Infirmières-Thiers,84,84007,Avignon,2023,21
...,...,...,...,...,...,...,...,...
48564,940460103,940460103,Centre Vert de Maison 3,94,94046,Maisons-Alfort,2023,21
48565,940460109,940460109,Centre Vert de Maison 9,94,94046,Maisons-Alfort,2023,21
48566,940460202,940460202,Alfort 2,94,94046,Maisons-Alfort,2023,21
48567,940540105,940540105,Carrières-Sentiers,94,94054,Orly,2023,21


In [147]:
apl_final = apl3.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl_final = apl_final[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl_final["year"]=year
apl_final

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,23,21,10010000,010010000,C,0.000000,99.511050,0.000000,1.0,0.000000,790.702997,19133.494274,790.702997,832.000000,L'Abergement-Clémenciat,1,01001,L'Abergement-Clémenciat
1,23,21,10020000,010020000,C,0.000000,82.294189,0.000000,1.0,0.000000,242.006732,8602.799088,242.006732,267.000000,L'Abergement-de-Varey,1,01002,L'Abergement-de-Varey
2,23,21,10040101,010040101,A,6.250000,237.132735,20.236235,1.0,20.236235,1524.501903,30885.192594,1524.501903,1796.951774,Les Pérouses-Triangle d'Activités,1,01004,Ambérieu-en-Bugey
3,23,21,10040102,010040102,A,8.500000,233.840987,29.470940,1.0,29.470940,3431.019785,28841.971324,3431.019785,3837.421806,Longeray-Gare,1,01004,Ambérieu-en-Bugey
4,23,21,10040201,010040201,A,9.916667,229.339553,34.161420,1.0,34.161420,4126.386485,29028.847779,4126.386485,4393.228271,Centre-Saint-Germain-Vareilles,1,01004,Ambérieu-en-Bugey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,23,21,2023560000,2B3560000,C,0.000000,0.000000,0.000000,1.0,0.000000,232.730248,284.294690,232.730248,148.000000,Zalana,202,2B356,Zalana
48565,23,21,2023610000,2B3610000,C,0.000000,81.856087,0.000000,1.0,0.000000,319.334308,1913.852047,319.334308,306.000000,Zilia,202,2B361,Zilia
48566,23,21,2023640000,2B3640000,C,0.000000,0.000000,0.000000,1.0,0.000000,61.443054,188.084826,61.443054,36.000000,Zuani,202,2B364,Zuani
48567,23,21,2023650000,2B3650000,C,0.000000,80.248989,0.000000,1.0,0.000000,154.968754,1225.315417,154.968754,100.000000,San-Gavino-di-Fiumorbo,202,2B365,San-Gavino-di-Fiumorbo


In [148]:
apl_final[apl_final["iris_string"]=="061070000"]

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
2499,23,21,61070000,061070000,C,0.0,153.359514,0.0,1.0,0.0,58.456659,893.24945,58.456659,65.0,La Roque-en-Provence,6,06107,La Roque-en-Provence


In [149]:
d6=apl_final[apl_final["dept"]==6]
d6

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
2095,23,21,60010000,060010000,C,0.000,31.653746,0.000000,1.0,0.000000,98.567009,308.542344,98.567009,93.000000,Aiglun,6,06001,Aiglun
2096,23,21,60020000,060020000,C,0.000,61.031106,0.000000,1.0,0.000000,65.059257,304.279731,65.059257,51.000000,Amirat,6,06002,Amirat
2097,23,21,60030000,060030000,C,0.000,75.126867,0.000000,1.0,0.000000,587.149107,1313.785817,587.149107,640.000000,Andon,6,06003,Andon
2098,23,21,60040101,060040101,C,4.000,285.766353,2.067246,1.0,2.067246,1473.977572,193494.106700,1473.977572,1763.364267,Les Trois Moulins,6,06004,Antibes
2099,23,21,60040102,060040102,A,2.000,228.518837,1.294370,1.0,1.294370,2385.234632,154515.319808,2385.234632,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,23,21,61610105,061610105,A,4.625,302.771368,2.231910,1.0,2.231910,4125.943504,207221.594154,4125.943504,3584.453834,Marina,6,06161,Villeneuve-Loubet
2591,23,21,61610106,061610106,A,6.000,215.973613,3.989227,1.0,3.989227,3881.138444,150405.076690,3881.138444,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
2592,23,21,61610107,061610107,A,5.000,241.982227,3.043800,1.0,3.043800,2600.798789,164268.335662,2600.798789,2766.654836,Plan,6,06161,Villeneuve-Loubet
2593,23,21,61620000,061620000,C,0.000,61.920370,0.000000,1.0,0.000000,942.142750,2245.326244,942.142750,719.000000,La Brigue,6,06162,La Brigue


In [150]:
d6["apl"].describe()
# Ca change à cause des cabinets multi-dept

count    500.000000
mean     198.186967
std      103.155984
min        0.000000
25%      122.413644
50%      202.088750
75%      284.766611
max      488.156909
Name: apl, dtype: float64

In [59]:
# apl_final.to_csv("apl_france.csv", index=False)

In [60]:
# apl_final["study_key"]=dico["key"]
# study.to_sql("apl_study", config.connection_string, schema="apl", if_exists="append", index=False)
# apl_final.to_sql("apl", config.connection_string, schema="apl", if_exists="append", index=False)

In [62]:
sum_pop = np.sum(apl_final["pop_gp"])
apl_final["meanw"]=apl_final["apl"]*apl_final["pop_gp"]*len(apl_final)/sum_pop
np.mean(apl_final["meanw"])

81.99000755635019